# Stacked Autoencoder Model

### Loading Data

In [1]:
import os
from skimage.feature import hog
from skimage import exposure
import numpy as np
from sklearn import svm
from sklearn.metrics import accuracy_score, confusion_matrix, roc_curve, roc_auc_score, precision_recall_curve, average_precision_score
import tensorflow as tf
from tensorflow.keras.layers.experimental import preprocessing
from tensorflow.keras.models import Model
import cv2
from scipy import stats
from sklearn.neighbors import LocalOutlierFactor


from tensorflow.keras.layers import Input, Conv2D, MaxPooling2D, Flatten, Dense, UpSampling2D, Flatten
from tensorflow.keras.applications import VGG16
from tensorflow.keras.layers import GlobalAveragePooling2D
from tensorflow.keras.models import Model
from sklearn.ensemble import IsolationForest
from sklearn.metrics import classification_report


import seaborn as sns
import matplotlib.pyplot as plt




In [2]:
from google.colab import drive
drive.mount('/content/drive', force_remount=True)

Mounted at /content/drive


In [3]:
base_path = '/content/drive/.shortcut-targets-by-id/1W01KwNde9fWkWs9206tNtZsdc3BVs97m/anomaly/mvtec_anomaly_detection/'


In [4]:
# Set random seeds for reproducibility
random_seed = 47

# Set the random seed for NumPy
np.random.seed(random_seed)

# Set the random seed for TensorFlow
tf.random.set_seed(random_seed)

In [5]:
# Specify the list of classes (folders)
classes = ["zipper", "wood", "transistor"]

In [6]:
def load_and_preprocess_image(image_path, target_size=(180, 180)):
    # Load the image using OpenCV
    image = cv2.imread(image_path)

    # Resize the image to the target size
    image = cv2.resize(image, target_size)

    # Normalize the pixel values to the [0, 1] range
    image = image.astype("float32") / 255.0

    return image

In [7]:
# Initialize lists to store images and labels
images_train = []
labels_train = []




In [8]:
images_test = []
labels_test = []


In [9]:
# Load and preprocess the images for the training dataset (only "good" class)
for class_name in classes:
    train_folder = os.path.join(base_path, class_name, "train", "good")

    for filename in os.listdir(train_folder):
        image_path = os.path.join(train_folder, filename)
        image = load_and_preprocess_image(image_path, target_size=(180, 180))
        images_train.append(image)
        labels_train.append(0)


In [10]:
print('Dimension of objects in the list images_train: ', images_train[0].shape)
print('Number of samples in the list images_train: ', len(images_train))



Dimension of objects in the list images_train:  (180, 180, 3)
Number of samples in the list images_train:  700


In [11]:
# Load and preprocess the images for the testing dataset (both "good" and "defected" classes)
for class_name in classes:
    test_folder = os.path.join(base_path, class_name, "test")

    for folder_name in os.listdir(test_folder):
        defect_folder = os.path.join(test_folder, folder_name)

        for filename in os.listdir(defect_folder):
            image_path = os.path.join(defect_folder, filename)
            image = load_and_preprocess_image(image_path, target_size=(180, 180))
            images_test.append(image)
            labels_test.append(0 if folder_name == "good" else 1)


In [12]:
print('Dimension of objects in the list images_test: ', images_test[0].shape)
print('Number of samples in the list images_test: ', len(images_test))


Dimension of objects in the list images_test:  (180, 180, 3)
Number of samples in the list images_test:  330


In [13]:
num_zeros = labels_test.count(0)
num_ones = labels_test.count(1)

print(f"Number of 0s in labels_test: {num_zeros}")
print(f"Number of 1s in labels_test: {num_ones}")

Number of 0s in labels_test: 111
Number of 1s in labels_test: 219


In [14]:
# Convert the data to numpy arrays
X_train = np.array(images_train)
X_test = np.array(images_test)
labels_train = np.array(labels_train)
labels_test = np.array(labels_test)


In [15]:
X_train[5].shape, X_test[0].shape

((180, 180, 3), (180, 180, 3))

In [16]:
# input shape
input_shape = (180, 180, 3)

# Simple Hybrid Model

In [19]:

# CNN model for feature extraction
input_layer = Input(shape=input_shape)
x = Conv2D(16, (3, 3), activation='relu')(input_layer)
x = MaxPooling2D((2, 2))(x)
x = Conv2D(32, (3, 3), activation='relu')(x)
x = MaxPooling2D((2, 2))(x)
x = Flatten()(x)
x = Dense(64, activation='relu')(x)
output_layer = Dense(1, activation='sigmoid')(x)

cnn_model = Model(inputs=input_layer, outputs=output_layer)


In [20]:
# Compile the CNN model
cnn_model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])


In [21]:
# Train the model
cnn_model.fit(X_train, labels_train, epochs=3, batch_size=32)


Epoch 1/3
22/22 [==============================] - 11s 32ms/step - loss: 0.0327 - accuracy: 0.9657
Epoch 2/3
22/22 [==============================] - 0s 16ms/step - loss: 1.2313e-18 - accuracy: 1.0000
Epoch 3/3
22/22 [==============================] - 0s 16ms/step - loss: 5.6538e-20 - accuracy: 1.0000


In [22]:
# Extract features using the CNN model from the full training set
X_train_features = cnn_model.predict(X_train)

22/22 [==============================] - 0s 8ms/step


In [23]:
# Train an Isolation Forest on the extracted features
iso_forest = IsolationForest(contamination=0.1)
iso_forest.fit(X_train_features)


IsolationForest(contamination=0.1)

In [24]:
# Extract features using the CNN model from the test set
X_test_features = cnn_model.predict(X_test)

11/11 [==============================] - 0s 18ms/step


In [25]:
# Predict anomalies using the Isolation Forest
anomaly_predictions = iso_forest.predict(X_test_features)

# Convert Isolation Forest predictions (-1 for anomalies, 1 for inliers) to binary (1 for anomalies, 0 for inliers)
binary_predictions = np.where(anomaly_predictions == -1, 1, 0)

In [26]:
# Evaluate the performance
print(classification_report(labels_test, binary_predictions))

              precision    recall  f1-score   support

           0       0.34      1.00      0.50       111
           1       0.00      0.00      0.00       219

    accuracy                           0.34       330
   macro avg       0.17      0.50      0.25       330
weighted avg       0.11      0.34      0.17       330



/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


# Hybrid model with Autoencoder

In [17]:
# Define a CNN-based autoencoder for feature extraction
def create_autoencoder(input_shape):
    input_img = Input(shape=input_shape)

    # Encoder
    x = Conv2D(64, (3, 3), activation='relu', padding='same')(input_img)
    x = MaxPooling2D((3, 3), padding='same')(x)
    x = Conv2D(32, (3, 3), activation='relu', padding='same')(x)
    x = MaxPooling2D((3, 3), padding='same')(x)

    # Decoder
    x = Conv2D(32, (3, 3), activation='relu', padding='same')(x)
    x = UpSampling2D((3, 3))(x)
    x = Conv2D(32, (3, 3), activation='relu', padding='same')(x)
    x = UpSampling2D((3, 3))(x)
    decoded = Conv2D(3, (3, 3), activation='sigmoid', padding='same')(x)

    autoencoder = Model(input_img, decoded)
    encoder = Model(input_img, x)  # Encoder can be used for feature extraction
    return autoencoder, encoder

In [18]:
autoencoder, encoder = create_autoencoder(input_shape)

autoencoder.compile(optimizer='adam', loss='mse')



In [19]:
autoencoder.fit(X_train, X_train, epochs=3, batch_size=16)


Epoch 1/3
44/44 [==============================] - 13s 39ms/step - loss: 0.0344
Epoch 2/3
44/44 [==============================] - 1s 24ms/step - loss: 0.0079
Epoch 3/3
44/44 [==============================] - 1s 24ms/step - loss: 0.0047


In [20]:
# Extract features using the encoder
X_train_features_auto = encoder.predict(X_train)

22/22 [==============================] - 1s 21ms/step


In [21]:
X_train_features_auto.shape

(700, 180, 180, 32)

In [22]:
X_train_features_auto_reshaped = X_train_features_auto.reshape(X_train_features_auto.shape[0], -1)

In [23]:
X_train_features_auto_reshaped.shape

(700, 1036800)

In [24]:
# Train an Isolation Forest on the extracted features
iso_forest_auto = IsolationForest(contamination=0.05)
iso_forest_auto.fit(X_train_features_auto_reshaped)


IsolationForest(contamination=0.05)

In [25]:
X_test_features_auto = encoder.predict(X_test)

11/11 [==============================] - 0s 30ms/step


In [26]:
X_test_features_auto.shape

(330, 180, 180, 32)

In [27]:
# Reshape X_test_features_auto to have only 2 dimensions
X_test_features_auto_reshaped = X_test_features_auto.reshape(X_test_features_auto.shape[0], -1)


In [28]:
X_test_features_auto_reshaped.shape

(330, 1036800)

In [29]:
anomaly_predictions_outo = iso_forest_auto.predict(X_test_features_auto_reshaped)

In [30]:
binary_predictions_auto = np.where(anomaly_predictions_outo == -1, 1, 0)

In [31]:
print(classification_report(labels_test, binary_predictions_auto))

              precision    recall  f1-score   support

           0       0.35      0.95      0.51       111
           1       0.79      0.11      0.19       219

    accuracy                           0.39       330
   macro avg       0.57      0.53      0.35       330
weighted avg       0.64      0.39      0.29       330



# Local outlier method applied to the same features

In [32]:
# Create a LOF model
lof = LocalOutlierFactor(n_neighbors=20, contamination=0.1)

In [33]:
# Fit the LOF model on X_train_features
lof.fit(X_train_features_auto_reshaped)

LocalOutlierFactor(contamination=0.1)

In [37]:
# Apply the trained LOF model to X_test_features
anomaly_labels = lof.fit_predict(X_test_features_auto_reshaped)

# Convert anomaly labels to anomaly scores
anomaly_scores = -lof.negative_outlier_factor_

In [38]:
# Define a threshold to classify anomalies (you can adjust this threshold)
threshold = -1.5

# Create binary labels where 1 indicates an anomaly (defective) and 0 indicates an inlier (normal)
defect_labels = (anomaly_scores < threshold).astype(int)

In [39]:
print(classification_report(labels_test, defect_labels))

              precision    recall  f1-score   support

           0       0.34      1.00      0.50       111
           1       0.00      0.00      0.00       219

    accuracy                           0.34       330
   macro avg       0.17      0.50      0.25       330
weighted avg       0.11      0.34      0.17       330



/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


# Hybrid Model with Pre-trained model

In [61]:
# Define a function to create a VGG16-based hybrid model
def create_hybrid_model(input_shape):
    base_model = VGG16(weights='imagenet', include_top=False, input_shape=input_shape)
    for layer in base_model.layers:
        layer.trainable = False
    x = GlobalAveragePooling2D()(base_model.output)
    x = Dense(1, activation='sigmoid')(x)
    model = Model(inputs=base_model.input, outputs=x)
    return model


In [62]:
# Create and train a hybrid model for one-class classification
hybrid_model = create_hybrid_model(input_shape=(180, 180, 3))
hybrid_model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])
hybrid_model.fit(X_train, labels_train, epochs=10, batch_size=32)


58889256/58889256 [==============================] - 3s 0us/step
Epoch 1/10
22/22 [==============================] - 9s 240ms/step - loss: 0.4504 - accuracy: 0.9314
Epoch 2/10
22/22 [==============================] - 2s 94ms/step - loss: 0.1844 - accuracy: 1.0000
Epoch 3/10
22/22 [==============================] - 2s 94ms/step - loss: 0.0967 - accuracy: 1.0000
Epoch 4/10
22/22 [==============================] - 2s 96ms/step - loss: 0.0623 - accuracy: 1.0000
Epoch 5/10
22/22 [==============================] - 2s 96ms/step - loss: 0.0448 - accuracy: 1.0000
Epoch 6/10
22/22 [==============================] - 2s 97ms/step - loss: 0.0345 - accuracy: 1.0000
Epoch 7/10
22/22 [==============================] - 2s 98ms/step - loss: 0.0276 - accuracy: 1.0000
Epoch 8/10
22/22 [==============================] - 2s 99ms/step - loss: 0.0227 - accuracy: 1.0000
Epoch 9/10
22/22 [==============================] - 2s 101ms/step - loss: 0.0191 - accuracy: 1.0000
Epoch 10/10
22/22 [=======================

In [63]:
# Extract features using the hybrid model
feature_extractor = Model(inputs=hybrid_model.input, outputs=hybrid_model.layers[-2].output)
X_train_features = feature_extractor.predict(X_train)

# Train an Isolation Forest on the extracted features
iso_forest_vgg = IsolationForest(contamination=0.05)  # Adjust the contamination parameter as needed
iso_forest_vgg.fit(X_train_features)


22/22 [==============================] - 2s 98ms/step


IsolationForest(contamination=0.05)

In [64]:
# Extract features using the feature extractor
X_test_features_vgg = feature_extractor.predict(X_test)

# Predict anomalies using the Isolation Forest
anomaly_predictions_vgg = iso_forest.predict(X_test_features)



11/11 [==============================] - 2s 242ms/step


In [65]:
# Convert Isolation Forest predictions (-1 for anomalies, 1 for inliers) to binary (0 for anomalies, 1 for inliers)
binary_predictions_vgg = np.where(anomaly_predictions == -1, 0, 1)

# Evaluate the performance
print(classification_report(labels_test, binary_predictions))

              precision    recall  f1-score   support

           0       0.34      1.00      0.50       111
           1       0.00      0.00      0.00       219

    accuracy                           0.34       330
   macro avg       0.17      0.50      0.25       330
weighted avg       0.11      0.34      0.17       330



/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
